In [2]:
import pandas as pd

In [6]:
df=pd.read_csv("C:\practise\day6\Employee_Dataset.csv")
print(df)

    employee_id department     designation  age  salary joining_date  \
0       EMP1000         IT  Senior Analyst   60   85000      invalid   
1         emp_1         IT         Analyst  NaN   55000   10/06/2020   
2       EMP1002      Sales             NaN   28   85000          NaN   
3       EMP1003        it          Analyst   45   35000      invalid   
4           NaN         IT             mgr  150   85000          NaN   
..          ...        ...             ...  ...     ...          ...   
995     EMP1995    Finance             NaN   60   55000   2019-05-10   
996     EMP1996         IT         Analyst   35  250000      invalid   
997     EMP1997      Sales  Senior Analyst  150  250000   2021/07/15   
998     emp_998    Finance             NaN   60   85000   2019-05-10   
999     EMP1999        it          Manager  150  ₹75000      invalid   

    last_promotion_date experience_years performance_rating is_active  
0            01/04/2021                5                  3    

In [8]:
#1. Convert joining_date to datetime and count how many rows failed conversion.
df['joining_date'] = pd.to_datetime(df['joining_date'], errors='coerce')
failed_conversions = df['joining_date'].isna().sum()
print(f"Number of rows that failed conversion: {failed_conversions}")

Number of rows that failed conversion: 377


C:\Users\Dolly\AppData\Local\Temp\ipykernel_22416\2255390756.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['joining_date'] = pd.to_datetime(df['joining_date'], errors='coerce')


In [9]:
#2. Clean employee_id and identify how many duplicate employees exist after standardization.
df['employee_id'] = df['employee_id'].str.strip().str.upper()
duplicate_count = df['employee_id'].duplicated().sum()
print(f"Number of duplicate employees after standardization: {duplicate_count}")

Number of duplicate employees after standardization: 503


In [11]:
#3. Standardize department & average salary per department
df["department_clean"] = df["department"].str.strip().str.title()

df["salary_num"] = pd.to_numeric(df["salary"], errors="coerce")

avg_salary = df[df["salary_num"].notna()] \
                .groupby("department_clean")["salary_num"].mean()

print(avg_salary)

department_clean
Finance    113253.012048
Hr         114800.000000
It         107944.444444
Sales      119000.000000
Name: salary_num, dtype: float64


In [13]:
#4. Convert age to numeric – valid salary but invalid age
df["age_num"] = pd.to_numeric(df["age"], errors="coerce")

invalid_age = df[df["salary_num"].notna() & df["age_num"].isna()]
print(invalid_age[["employee_id", "age", "salary"]])

    employee_id      age  salary
1         EMP_1      NaN   55000
30      EMP1030      NaN   85000
31          NaN      NaN   55000
38       EMP_38      NaN   35000
45          NaN      NaN  120000
..          ...      ...     ...
959         NaN  unknown  120000
974     EMP_974  unknown   85000
976     EMP1976  unknown  250000
982         NaN  unknown   55000
991         NaN      NaN  120000

[120 rows x 3 columns]


In [14]:
#Clean salary & detect outliers using IQR
df["salary_clean"] = pd.to_numeric(df["salary"], errors="coerce")
Q1 = df["salary_clean"].quantile(0.25)
Q3 = df["salary_clean"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = df[(df["salary_clean"] < lower_bound) | (df["salary_clean"] > upper_bound)]
print(outliers[["employee_id", "salary_clean"]])

    employee_id  salary_clean
11          NaN      250000.0
16          NaN      250000.0
25          NaN      250000.0
34          NaN      250000.0
41          NaN      250000.0
..          ...           ...
955         NaN      250000.0
969     EMP1969      250000.0
976     EMP1976      250000.0
996     EMP1996      250000.0
997     EMP1997      250000.0

[136 rows x 2 columns]


In [15]:
#6. Performance rating → numeric & median per designation
df["performance_rating_num"] = pd.to_numeric(df["performance_rating"], errors="coerce")
median_rating = df[df["performance_rating_num"].notna()] \
                .groupby("designation")["performance_rating_num"].median()
print(median_rating)

designation
ANALYST           3.0
Analyst           3.0
Manager           3.0
Senior Analyst    3.0
mgr               3.0
Name: performance_rating_num, dtype: float64


In [16]:
#7. Last promotion earlier than joining date
df["last_promotion_date"] = pd.to_datetime(df["last_promotion_date"], errors="coerce")
invalid_promotion = df[df["last_promotion_date"] < df["joining_date"]]
print(invalid_promotion[["employee_id", "joining_date", "last_promotion_date"]])

    employee_id joining_date last_promotion_date
24      EMP1024   2021-07-15          2021-01-04
40       EMP_40   2021-07-15          2021-01-04
44       EMP_44   2021-07-15          2021-01-04
47          NaN   2021-07-15          2021-01-04
71       EMP_71   2021-07-15          2021-01-04
82       EMP_82   2021-07-15          2021-01-04
85          NaN   2021-07-15          2021-01-04
124         NaN   2021-07-15          2021-01-04
138     EMP_138   2021-07-15          2021-01-04
169     EMP_169   2021-07-15          2021-01-04
187         NaN   2021-07-15          2021-01-04
191     EMP1191   2021-07-15          2021-01-04
200         NaN   2021-07-15          2021-01-04
243         NaN   2021-07-15          2021-01-04
247         NaN   2021-07-15          2021-01-04
269         NaN   2021-07-15          2021-01-04
275         NaN   2021-07-15          2021-01-04
293     EMP1293   2021-07-15          2021-01-04
331     EMP_331   2021-07-15          2021-01-04
360     EMP_360   20

In [17]:
#8. Experience cleaned & mismatches (experience > age)
df["experience_years"] = pd.to_numeric(df["experience_years"], errors="coerce")
invalid_experience = df[df["experience_years"] > df["age_num"]]
print(invalid_experience[["employee_id", "age_num", "experience_years"]])

    employee_id  age_num  experience_years
10       EMP_10     -5.0               8.0
17       EMP_17     -5.0               3.0
23       EMP_23     -5.0              -2.0
43          NaN     -5.0               1.0
49          NaN     -5.0               5.0
..          ...      ...               ...
965         NaN     -5.0              12.0
981         NaN     -5.0              -2.0
985     EMP_985     -5.0              -2.0
990     EMP_990     -5.0              12.0
994     EMP1994     -5.0              12.0

[106 rows x 3 columns]


In [20]:
#9. Standardize designation & count active employees
df["designation_clean"] = df["designation"].str.strip().str.title()

active_count = df[df["is_active"] == True] \
                .groupby("designation_clean").size()

print(active_count)


Series([], dtype: int64)


In [23]:
#10. Convert is_active to boolean & find inactive with recent promotions
df["is_active_bool"] = df["is_active"].astype(bool)
recent_promotion_inactive = df[(df["is_active_bool"] == False) & (df["last_promotion_date"] > pd.to_datetime("2023-01-01"))]
print(recent_promotion_inactive[["employee_id", "is_active_bool", "last_promotion_date"]])  


Empty DataFrame
Columns: [employee_id, is_active_bool, last_promotion_date]
Index: []


In [25]:
#11.Employee tenure & above 90th percentile
today = pd.to_datetime("today")

df["tenure_years"] = (today - df["joining_date_dt"]).dt.days / 365

p90 = df["tenure_years"].quantile(0.90)

senior = df[df["tenure_years"] > p90]
print(senior[["employee_id", "tenure_years"]])


Empty DataFrame
Columns: [employee_id, tenure_years]
Index: []


In [27]:
#12. Departments with >25% invalid salary
invalid_salary = df["salary_num"].isna()

dept_invalid = df.groupby("department_clean")["salary_num"] \
                 .apply(lambda x: x.isna().mean())

problem_dept = dept_invalid[dept_invalid > 0.25]
print(problem_dept)

department_clean
Finance    0.366412
Hr         0.321267
It         0.340659
Sales      0.398340
Name: salary_num, dtype: float64


In [30]:
#13. High performance (≥4) but below-median salary
median_sal = df["salary_num"].median()

flag = df[
    (df["performance_rating_num"] >= 4) &
    (df["salary_num"] < median_sal)
]

print(flag[["employee_id", "salary_num", "performance_rating_num"]])


    employee_id  salary_num  performance_rating_num
21          NaN     55000.0                     4.0
29          NaN     55000.0                     4.0
31          NaN     55000.0                     4.0
66          NaN     35000.0                     4.0
92          NaN     35000.0                     5.0
118         NaN     35000.0                     4.0
145     EMP1145     35000.0                     4.0
149     EMP1149     35000.0                     5.0
189         NaN     55000.0                     4.0
208     EMP_208     55000.0                     5.0
212         NaN     55000.0                     5.0
312     EMP1312     55000.0                     4.0
314         NaN     55000.0                     4.0
320         NaN     35000.0                     4.0
327     EMP1327     35000.0                     4.0
335     EMP_335     55000.0                     5.0
354         NaN     35000.0                     4.0
366     EMP1366     55000.0                     5.0
382         

In [31]:
#14. No promotion date & experience > 5 years
no_promo_exp = df[df["last_promotion_date"].isna() & (df["experience_years"] > 5)]
print(no_promo_exp[["employee_id", "experience_years", "last_promotion_date"]])

    employee_id  experience_years last_promotion_date
4           NaN              12.0                 NaT
5           NaN               8.0                 NaT
6           NaN              12.0                 NaT
10       EMP_10               8.0                 NaT
12       EMP_12               8.0                 NaT
..          ...               ...                 ...
977     EMP_977               8.0                 NaT
978     EMP1978               8.0                 NaT
993     EMP1993              12.0                 NaT
994     EMP1994              12.0                 NaT
998     EMP_998              12.0                 NaT

[191 rows x 3 columns]


In [35]:
df["joining_date_dt"] = pd.to_datetime(df["joining_date"], errors="coerce")
df["last_promotion_date"] = pd.to_datetime(df["last_promotion_date"], errors="coerce")
df["salary_num"] = pd.to_numeric(df["salary"], errors="coerce")
df["performance_rating_num"] = pd.to_numeric(df["performance_rating"], errors="coerce")
df["age_num"] = pd.to_numeric(df["age"], errors="coerce")
df["experience_years"] = pd.to_numeric(df["experience_years"], errors="coerce")

df["is_active_bool"] = df["is_active"].astype(bool)

In [ ]:
#1.Promotion gap < 1 year
gap = (df["last_promotion_date"] - df["joining_date_dt"]).dt.days

df["promo_less_1yr"] = gap < 365

print(df[df["promo_less_1yr"]])


    employee_id department     designation      age  salary joining_date  \
9       EMP1009         It             NaN       22  120000   2020-10-06   
24      EMP1024    Finance             mgr       35   85000   2021-07-15   
40       EMP_40        NaN             NaN       60   85000   2021-07-15   
44       EMP_44      Sales             mgr       45  ₹75000   2021-07-15   
47          NaN    Finance         Analyst  unknown  250000   2021-07-15   
..          ...        ...             ...      ...     ...          ...   
943     EMP1943    Finance             mgr       45   35000   2020-10-06   
945         NaN         Hr         Manager       -5   35000   2020-10-06   
966         NaN         It         Analyst       45     NaN   2020-10-06   
974     EMP_974        NaN             mgr  unknown   85000   2021-07-15   
975         NaN        NaN  Senior Analyst       28   55000   2020-10-06   

    last_promotion_date  experience_years performance_rating is_active  \
9            

In [ ]:
#2.Salary > dept avg BUT rating < dept median
dept_avg_sal = df.groupby("department_clean")["salary_num"].transform("mean")
dept_med_rat = df.groupby("department_clean")["performance_rating_num"].transform("median")

cond = (df["salary_num"] > dept_avg_sal) & \
       (df["performance_rating_num"] < dept_med_rat)

print(df[cond][["employee_id","salary_num","performance_rating_num"]])


    employee_id  salary_num  performance_rating_num
45          NaN    120000.0                     1.0
115     EMP_115    250000.0                     2.0
127         NaN    120000.0                     1.0
157         NaN    250000.0                     1.0
169     EMP_169    120000.0                     2.0
173     EMP_173    120000.0                     1.0
217         NaN    250000.0                     2.0
223         NaN    120000.0                     1.0
270         NaN    120000.0                     2.0
273         NaN    120000.0                     2.0
286         NaN    120000.0                     1.0
288         NaN    250000.0                     1.0
297     EMP_297    120000.0                     2.0
300     EMP1300    120000.0                     2.0
346         NaN    250000.0                     2.0
372     EMP1372    250000.0                     2.0
386         NaN    250000.0                     2.0
401     EMP1401    250000.0                     1.0
418         

In [ ]:
#3.Age band + count per department
df["age_band"] = pd.cut(df["age_num"], bins=[0, 30, 40, 50, 60, 100], labels=["<30", "30-39", "40-49", "50-59", "60+"])
band_count = df.groupby(["department_clean", "age_band"]).size().unstack(fill_value=0)
print(band_count)


age_band          <30  30-39  40-49  50-59  60+
department_clean                               
Finance            33     16     15     13    0
Hr                 54     26     26     22    0
It                 53     36     42     37    0
Sales              48     24     33     25    0


C:\Users\Dolly\AppData\Local\Temp\ipykernel_22416\438635858.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  band_count = df.groupby(["department_clean", "age_band"]).size().unstack(fill_value=0)


In [40]:
#4.Departments where avg experience > company avg
company_avg = df["experience_years"].mean()
dept_avg_exp = df.groupby("department_clean")["experience_years"].mean()
high_exp_dept = dept_avg_exp[dept_avg_exp > company_avg]
print(high_exp_dept)

department_clean
Finance    4.826923
Sales      4.709677
Name: experience_years, dtype: float64


In [42]:
#5.Inactive but promotion within last 2 years
two_years = pd.to_datetime("today") - pd.DateOffset(years=2)

cond = (~df["is_active_bool"]) & \
       (df["last_promotion_date"] >= two_years)

print(df[cond])

Empty DataFrame
Columns: [employee_id, department, designation, age, salary, joining_date, last_promotion_date, experience_years, performance_rating, is_active, joining_date_dt, department_clean, salary_num, age_num, salary_clean, performance_rating_num, designation_clean, is_active_bool, tenure_years, promo_less_1yr, age_band]
Index: []

[0 rows x 21 columns]


In [45]:
#6.Designations with >20% invalid experience
invalid_exp_ratio = df.groupby("designation_clean")["experience_years"] \
                      .apply(lambda x: x.isna().mean())

bad_design = invalid_exp_ratio[invalid_exp_ratio > 0.20]

print(bad_design)

designation_clean
Mgr               0.264045
Senior Analyst    0.202128
Name: experience_years, dtype: float64


In [46]:
#7.Salary / experience ratio – 95th percentile
df["sal_exp_ratio"] = df["salary_num"] / df["experience_years"]
p95 = df["sal_exp_ratio"].quantile(0.95)
extreme= df[df["sal_exp_ratio"] > p95]
print(extreme[["employee_id", "sal_exp_ratio"]])

    employee_id  sal_exp_ratio
124         NaN       250000.0
133         NaN       250000.0
176         NaN       250000.0
224         NaN       250000.0
262         NaN       250000.0
399         NaN       250000.0
401     EMP1401       250000.0
470     EMP1470       250000.0
525     EMP_525       250000.0
529     EMP_529       250000.0
576         NaN       250000.0
667     EMP_667       250000.0
821     EMP_821       250000.0
848         NaN       250000.0
862         NaN       250000.0
976     EMP1976       250000.0


In [49]:
#8.Age valid but inconsistent with experience
cond = df[(df["age_num"].notna()) & (df["experience_years"].notna()) & (df["experience_years"] > df["age_num"])]
print(cond[["employee_id", "age_num", "experience_years"]])

    employee_id  age_num  experience_years
10       EMP_10     -5.0               8.0
17       EMP_17     -5.0               3.0
23       EMP_23     -5.0              -2.0
43          NaN     -5.0               1.0
49          NaN     -5.0               5.0
..          ...      ...               ...
965         NaN     -5.0              12.0
981         NaN     -5.0              -2.0
985     EMP_985     -5.0              -2.0
990     EMP_990     -5.0              12.0
994     EMP1994     -5.0              12.0

[106 rows x 3 columns]


In [50]:
#9.Department-wise attrition rate
attrition = df.groupby("department_clean")["is_active_bool"].apply(lambda x: 1 - x.mean())
print(attrition)

department_clean
Finance    0.0
Hr         0.0
It         0.0
Sales      0.0
Name: is_active_bool, dtype: float64


In [52]:
#10.Final quality score per row
df["valid_fields"] = (
    df["salary_num"].notna().astype(int) +
    df["age_num"].notna().astype(int) +
    df["experience_years"].notna().astype(int) +
    df["performance_rating_num"].notna().astype(int) +
    df["joining_date_dt"].notna().astype(int)
)

df["quality_flag"] = df["valid_fields"] < 4

print(df[["employee_id","valid_fields","quality_flag"]])

    employee_id  valid_fields  quality_flag
0       EMP1000             4         False
1         EMP_1             4         False
2       EMP1002             4         False
3       EMP1003             4         False
4           NaN             4         False
..          ...           ...           ...
995     EMP1995             5         False
996     EMP1996             4         False
997     EMP1997             4         False
998     EMP_998             4         False
999     EMP1999             2          True

[1000 rows x 3 columns]


In [ ]:
#15.flag rows violating at least two constraints
df["constraint_violations"] = 5 - df["valid_fields"]
df["flagged"] = df["constraint_violations"] >= 2
print(df[["employee_id", "valid_fields", "constraint_violations", "flagged"]])

    employee_id  valid_fields  constraint_violations  flagged
0       EMP1000             4                      1    False
1         EMP_1             4                      1    False
2       EMP1002             4                      1    False
3       EMP1003             4                      1    False
4           NaN             4                      1    False
..          ...           ...                    ...      ...
995     EMP1995             5                      0    False
996     EMP1996             4                      1    False
997     EMP1997             4                      1    False
998     EMP_998             4                      1    False
999     EMP1999             2                      3     True

[1000 rows x 4 columns]


In [54]:
c1 = df["salary_num"].isna()
c2 = df["age_num"].isna()
c3 = df["last_promotion_date"] < df["joining_date_dt"]
c4 = df["experience_years"] > df["age_num"]

df["violation_count"] = c1.astype(int) + c2.astype(int) + c3.astype(int) + c4.astype(int)

df["flag_multiple_violations"] = df["violation_count"] >= 2

print(df[df["flag_multiple_violations"]])


    employee_id department     designation      age  salary joining_date  \
5           NaN         It             mgr      NaN  ₹75000   2019-05-10   
17       EMP_17      Sales  Senior Analyst       -5     NaN          NaT   
18       EMP_18    Finance  Senior Analyst      NaN     NaN          NaT   
44       EMP_44      Sales             mgr       45  ₹75000   2021-07-15   
47          NaN    Finance         Analyst  unknown  250000   2021-07-15   
..          ...        ...             ...      ...     ...          ...   
954         NaN         Hr         Analyst      NaN  ₹75000   2019-05-10   
962     EMP_962      Sales         Analyst      NaN     NaN   2020-10-06   
965         NaN      Sales             mgr       -5     NaN   2020-10-06   
974     EMP_974        NaN             mgr  unknown   85000   2021-07-15   
985     EMP_985      Sales             mgr       -5     NaN          NaT   

    last_promotion_date  experience_years performance_rating is_active  ...  \
5       